Hello fellow Rakamin students.

Stakeholder: **Trav#loka, Tik#t.com**

Anyway, the problem can be as straightforward as predicting if a consumer will take the product or not, or we can make it a little bit more complex, as "will the consumer take the product, **after** a marketing pitch?". So it's basically a bayesian probability. Not only P(take product) but P(take product | marketing pitch).

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.style as style
import matplotlib.ticker as mtick
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix, precision_recall_curve, auc, classification_report
from sklearn.pipeline import Pipeline
from scipy.stats import chi2_contingency
from sklearn.feature_selection import f_classif
from sklearn.metrics import ConfusionMatrixDisplay

style.use('fivethirtyeight')

In [2]:
df = pd.read_csv('../input/holiday-package-purchase-prediction/Travel.csv')
df.head()

In [3]:
# To check the independence of rows
df.duplicated(subset='CustomerID').any()

In [4]:
# We don't want to predict P(take prod), but P(take prod | company invited), because this is where the ineffective marketing cost come from
df_mkt = df[df['TypeofContact'] == 'Company Invited']

# Drop some looking-forward leakage
X = df_mkt.drop(['CustomerID','ProdTaken', 'TypeofContact', 'PitchSatisfactionScore', 'ProductPitched', 'DurationOfPitch'], axis = 1)
y = df_mkt['ProdTaken']

In [5]:
df_mkt.isna().sum().to_frame().T

In [6]:
df_mkt.shape

My plan is to predict whether a consumer will take the product or no if we gave them a marketing pitch, and then try to decrease ineffective marketing pitch & cost.
* Effective marketing pitch -> TypeofContact = "Company Invited",  and consumer take the product
* Ineffective marketing pitch -> TypeofContact = "Company Invited", and have a long "DurationofPitch" but the consumer don't take the product nonetheless.

For simplicity, I will only use Logistic Regression and Tree Based Algorithm.

# Some general exploration

Because some of the variables are interesting to explore.

In [7]:
# Create categorical df ProdTaken for easier interpretation
df['ProdTaken_YN'] = df['ProdTaken'].apply(lambda x: 'Yes' if x == 1 else 'No')

**Type of contact**

In [8]:
g = df['TypeofContact'].value_counts().plot(kind='barh')

for p in g.patches:
    g.annotate(format(p.get_width(), '0'),
               xy = (p.get_x() + p.get_width() / 2,
                     p.get_y() + p.get_height() / 2,),
               ha = 'center',
               va = 'center',
               color = 'white'
              )

In [9]:
g = df.groupby('TypeofContact')['ProdTaken_YN'].value_counts(normalize=True).unstack('ProdTaken_YN').plot(kind='barh', stacked=True)
g.xaxis.set_major_formatter(mtick.PercentFormatter(1.0))
g.set_ylabel('TypeofContact')
g.legend(loc='center', title = "ProdTaken?")

for p in g.patches:
    g.annotate(format(p.get_width(), '.2%'),
               xy = (p.get_x() + p.get_width() / 2,
                     p.get_y() + p.get_height() / 2,),
               ha = 'center',
               va = 'center',
               color = 'white'
              )

Okay. So, only 21.85% individual that's invited by the company actually taken the offer.

**Duration of Pitch**

In [10]:
df['DurationOfPitch'].describe().to_frame().T

In [11]:
sns.boxplot(data = df[df['DurationOfPitch'] < 60], x = 'DurationOfPitch', y = 'ProdTaken_YN')

In [12]:
sns.boxplot(data = df[df['DurationOfPitch'] < 60], x = 'DurationOfPitch', y = 'ProdTaken_YN', hue = 'TypeofContact')

Maybe short duration of pitch means that the consumer isn't really interested with the product offered. But it's a weak assumption because 75% of people, either they take or not the product, have duration of pitch < 20 minutes. But, the longer the duration of pitch, the higher the marketing cost, right?

**Number Of Followup**

In [13]:
g = df.groupby('NumberOfFollowups')['ProdTaken_YN'].value_counts(normalize=True).unstack('ProdTaken_YN').plot(kind='barh', stacked=True)
g.xaxis.set_major_formatter(mtick.PercentFormatter(1.0))
g.set_ylabel('Pitch Followup')
g.legend(loc='center', title = "ProdTaken?")

for p in g.patches:
    g.annotate(format(p.get_width(), '.2%'),
               xy = (p.get_x() + p.get_width() / 2,
                     p.get_y() + p.get_height() / 2,),
               ha = 'center',
               va = 'center',
               color = 'white'
              )

Higher followups -> Higher chance the customer will take the product!

**Pitch Satisfaction Score**

In [14]:
g = df.groupby('PitchSatisfactionScore')['ProdTaken_YN'].value_counts(normalize=True).unstack('ProdTaken_YN').plot(kind='barh', stacked=True)
g.xaxis.set_major_formatter(mtick.PercentFormatter(1.0))
g.set_ylabel('Pitch Rating')
g.legend(loc='center', title = "ProdTaken?")

for p in g.patches:
    g.annotate(format(p.get_width(), '.2%'),
               xy = (p.get_x() + p.get_width() / 2,
                     p.get_y() + p.get_height() / 2,),
               ha = 'center',
               va = 'center',
               color = 'white'
              )

In [15]:
g = sns.boxplot(data = df, y = 'PitchSatisfactionScore', x = 'NumberOfFollowups')
g.set_ylabel('Pitch Rating')

Not much things we can do. We'll start to model.

# Feature Selection

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42, stratify = y)

cat_cols = X_train.select_dtypes(include = 'object').copy().columns
num_cols = X_train.select_dtypes(include = 'number').copy().columns

X_train.shape

Okay, we have a really small dataset, so, it's better for our features also be minimal. It's important especially when we're selecting our categorical features, because maybe we wanted to one hot encode it later. 1135 independent rows, I think 10 features is already high.

In [17]:
cat_cols

In [18]:
num_cols

In [19]:
X_train[cat_cols].nunique().sort_values().to_frame().T

In [20]:
# define an empty dictionary to store chi-squared test results
chi2_check = {}

# loop over each column in the training set to calculate chi-statistic with the target variable
for column in cat_cols:
    chi, p, dof, ex = chi2_contingency(pd.crosstab(y_train, X_train[column]))
    chi2_check.setdefault('Feature',[]).append(column)
    chi2_check.setdefault('p-value',[]).append(round(p, 10))

# convert the dictionary to a DF
chi2_result = pd.DataFrame(data = chi2_check)
# chi2_result.sort_values(by = ['p-value'], ascending = True, ignore_index = True, inplace = True)
chi2_result.merge(X_train[cat_cols].describe().T.reset_index(), left_on ='Feature', right_on = 'index').sort_values(by = ['p-value', 'unique'])

In [21]:
selected_cat_cols = ['MaritalStatus', 'Designation', 'Occupation']

In [22]:
X_train[num_cols].nunique().sort_values().to_frame().T

In [23]:
fig = plt.figure(figsize = (10,6))
sns.heatmap(X_train[num_cols].corr(), annot=True)

In [24]:
# since f_class_if does not accept missing values, we will do a very crude imputation of missing values
# Calculate F Statistic and corresponding p values
F_statistic, p_values = f_classif(X_train[num_cols].fillna(X_train[num_cols].median()), y_train)
# convert to a DF
ANOVA_F_table = pd.DataFrame(data = {'Numerical_Feature': num_cols, 'F-Score': F_statistic, 'p values': p_values.round(decimals=10)})
ANOVA_F_table.merge(X_train[num_cols].describe().T.reset_index(), left_on = 'Numerical_Feature', right_on = 'index').sort_values(['F-Score', 'count'], ascending=False)

The score makes me wonder if the pitching simply failed because the target don't have a passport.............

Anyway, from the cardinality of our categorical feature, we already obtained 9 columns, and the lowest P-value for our categorical feature is 0.000001 or 1e-6. So I think we can only pick the top 3 numerical values (Passport, Age, MonthlyIncome)

In [25]:
selected_num_cols = ['Passport', 'Age', 'MonthlyIncome']
selected_cols = selected_cat_cols + selected_num_cols

# Feature Engineering

In [26]:
X_train[selected_cols].isna().sum()

In [27]:
X_test[selected_cols].isna().sum()

In [28]:
from sklearn.impute import SimpleImputer

numimputer = SimpleImputer(strategy = 'median')

X_train[selected_num_cols] = numimputer.fit_transform(X_train[selected_num_cols])
X_test[selected_num_cols] = numimputer.transform(X_test[selected_num_cols])

In [29]:
pd.get_dummies(X_train[selected_cat_cols], prefix_sep = ':')

In [30]:
X_trainf = pd.concat([X_train[selected_num_cols], pd.get_dummies(X_train[selected_cat_cols], prefix_sep = ':')], axis = 1)
X_testf = pd.concat([X_test[selected_num_cols], pd.get_dummies(X_test[selected_cat_cols], prefix_sep = ':')], axis = 1)

In [31]:
def data_processing(df):
    dfx = df.copy()
    dfx[selected_num_cols] = numimputer.transform(dfx[selected_num_cols])
    num = dfx[selected_num_cols]
    cat = pd.get_dummies(dfx[selected_cat_cols], prefix_sep = ':')
    return pd.concat([num, cat], axis = 1)

# Modelling

First, we will check the current performance of our marketing. So much inefficiencies right?

In [32]:
currentperform = y_test.to_frame()
currentperform['predict'] = 1
ConfusionMatrixDisplay.from_predictions(currentperform['ProdTaken'], currentperform['predict'])

Objective:
* TP > FP because False positive -> Ineffective marketing
* TP > FN because False negative -> So much opportunity wasted!

Ah yeah, because our dataset is kinda low. So, it's only about 75 independent events per 1 feature

In [33]:
X_trainf.shape[0] / X_trainf.shape[1]

In [34]:
# define modeling pipeline
reg = LogisticRegression(max_iter=1000, class_weight = 'balanced')

# define cross-validation criteria. RepeatedStratifiedKFold automatially takes care of the class imbalance while splitting
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)

# fit and evaluate the logistic regression pipeline with cross-validation as defined in cv
scores = cross_val_score(reg, X_trainf, y_train, scoring = 'roc_auc', cv = cv)
AUROC = np.mean(scores)

# print the mean AUROC score
print('Mean AUROC: %.4f' % (AUROC))

In [35]:
reg.fit(X_trainf, y_train)
print('success')

In [36]:
y_hat_test = reg.predict(X_testf)
ConfusionMatrixDisplay.from_predictions(y_test, y_hat_test)

High false positive? Hmm. It means that with logistic regression the marketing is still a little bit ineffective. Let's try using **tree based algorithm**.

In [37]:
rf = RandomForestClassifier()
rf.fit(X_trainf, y_train)
print('success')

In [38]:
scores = cross_val_score(rf, X_trainf, y_train, scoring = 'roc_auc', cv = cv)
AUROC = np.mean(scores)

# print the mean AUROC score
print('Mean AUROC: %.4f' % (AUROC))

In [39]:
y_hat_test = rf.predict(X_testf)
ConfusionMatrixDisplay.from_predictions(y_test, y_hat_test)

Better. But still, the false negative is kinda high, means we missed too much opportunity. 
* Maybe it's because of class imbalance. We will try use balanced weight in our algoritm.

In [40]:
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X_trainf, y_train)

scores = cross_val_score(xgb, X_trainf, y_train, scoring = 'roc_auc', cv = cv)
AUROC = np.mean(scores)

# print the mean AUROC score
print('Mean AUROC: %.4f' % (AUROC))

In [41]:
xgb = XGBClassifier(learning_rate = 0.34, objective = 'binary:logistic', eval_metric = 'aucpr', random_state = 42)
xgb.fit(X_trainf, y_train)
y_hat_test = xgb.predict(X_testf)
ConfusionMatrixDisplay.from_predictions(y_test, y_hat_test)

In [42]:
print(classification_report(y_test, y_hat_test))

In [43]:
# make preditions on our test set
y_hat_test = xgb.predict(X_testf)
# get the predicted probabilities
y_hat_test_proba = xgb.predict_proba(X_testf)
# select the probabilities of only the positive class (class 1 - default) 
y_hat_test_proba = y_hat_test_proba[:][: , 1]
# we will now create a new DF with actual classes and the predicted probabilities
# create a temp y_test DF to reset its index to allow proper concaternation with y_hat_test_proba
y_test_temp = y_test.copy()
y_test_temp.reset_index(drop = True, inplace = True)
y_test_proba = pd.concat([y_test_temp, pd.DataFrame(y_hat_test_proba)], axis = 1)
# Rename the columns
y_test_proba.columns = ['y_test_class_actual', 'y_hat_test_proba']
# Makes the index of one dataframe equal to the index of another dataframe.
y_test_proba.index = X_testf.index

In [44]:
# get the values required to plot a ROC curve
fpr, tpr, thresholds = roc_curve(y_test_proba['y_test_class_actual'], y_test_proba['y_hat_test_proba'])
# plot the ROC curve
plt.plot(fpr, tpr)
# plot a secondary diagonal line, with dashed line style and black color to represent a no-skill classifier
plt.plot(fpr, fpr, linestyle = '--', color = 'k')
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve');

In [45]:
# Calculate the Area Under the Receiver Operating Characteristic Curve (AUROC) on our test set
AUROC = roc_auc_score(y_test_proba['y_test_class_actual'], y_test_proba['y_hat_test_proba'])
AUROC

In [46]:
# draw a PR curve
# calculate the no skill line as the proportion of the positive class
no_skill = len(y_test[y_test == 1]) / len(y)
# plot the no skill precision-recall curve
plt.plot([0, 1], [no_skill, no_skill], linestyle='--', label='No Skill')

# calculate inputs for the PR curve
precision, recall, thresholds = precision_recall_curve(y_test_proba['y_test_class_actual'], y_test_proba['y_hat_test_proba'])
# plot PR curve
plt.plot(recall, precision, marker='.', label='Logistic')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.legend()
plt.title('PR curve');

In [47]:
# calculate PR AUC
auc_pr = auc(recall, precision)
auc_pr

**why after recall > 80% the performance nose dive?**

The curve smells like outlier + imbalance dataset. Let's try oversample... Not much of expectation though.

In [48]:
train_check = X_train.describe().T
train_check['IQR'] = train_check['75%'] - train_check['25%']
train_check['left_out'] = train_check['25%'] - 1.5 * train_check['IQR']
train_check['right_out'] = train_check['75%'] + 1.5 * train_check['IQR']
train_check[['mean', '50%', 'min', 'left_out', 'max', 'right_out']]

In [49]:
sns.boxplot(data = df, x = 'Age', y = 'ProdTaken_YN', hue = 'TypeofContact')

In [50]:
sns.boxplot(data = df, x = 'NumberOfTrips', y = 'ProdTaken_YN', hue = 'TypeofContact')

In [51]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(X_trainf, y_train)

In [52]:
xgb2 = XGBClassifier(learning_rate = 0.31, objective = 'binary:logistic', eval_metric = 'aucpr', random_state = 42)
xgb2.fit(X_resampled, y_resampled)
y_hat_test2 = xgb2.predict(X_testf)
ConfusionMatrixDisplay.from_predictions(y_test, y_hat_test2)

In [53]:
print(classification_report(y_test, y_hat_test2))

In [54]:
y_hat_test_proba2 = xgb2.predict_proba(X_testf)
# select the probabilities of only the positive class (class 1 - default) 
y_hat_test_proba2 = y_hat_test_proba2[:][: , 1]
# we will now create a new DF with actual classes and the predicted probabilities
# create a temp y_test DF to reset its index to allow proper concaternation with y_hat_test_proba
y_test_temp = y_test.copy()
y_test_temp.reset_index(drop = True, inplace = True)
y_test_proba2 = pd.concat([y_test_temp, pd.DataFrame(y_hat_test_proba2)], axis = 1)
# Rename the columns
y_test_proba2.columns = ['y_test_class_actual', 'y_hat_test_proba']
# Makes the index of one dataframe equal to the index of another dataframe.
y_test_proba2.index = X_testf.index

In [55]:
# draw a PR curve
# calculate the no skill line as the proportion of the positive class
no_skill = len(y_test[y_test == 1]) / len(y)
# plot the no skill precision-recall curve
plt.plot([0, 1], [no_skill, no_skill], linestyle='--', label='No Skill')
# calculate inputs for the PR curve
precision, recall, thresholds = precision_recall_curve(y_test_proba2['y_test_class_actual'], y_test_proba2['y_hat_test_proba'])
# plot PR curve
plt.plot(recall, precision, marker='.', label='Logistic')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.legend()
plt.title('PR curve');

Well... A little bit better. Maybe we should try undersample? But the data is too low. So, no.

Agh, don't have anymore idea with the knowledge I have right now! Any feedback on this (how to stabilize the PR curve) will be much appreciated!

Problem:
* Recall > 0.8 will make the precision nosedive

**Threshold tuning (to maximize recall)**

In [56]:
tr = 0.4
# crate a new column for the predicted class based on predicted probabilities and threshold
y_test_proba['y_test_class_predicted'] = np.where(y_test_proba['y_hat_test_proba'] > tr, 1, 0)
# create the confusion matrix
ConfusionMatrixDisplay.from_predictions(y_test, y_test_proba['y_test_class_predicted'])

In [57]:
print(classification_report(y_test, y_test_proba['y_test_class_predicted']))

Okay, this is the best that I can do right now.

# How the model can be used?

Based on the precision recall score:
* We can slash 94% of the ineffective marketing cost
* The model can predict up to 81% of the available potential customer to be pitched, 
* Precision of 0.79 -> means that every 5 potential customer predicted, there is 1/5 chance that the one predicted actually isn't potential.

In [58]:
def last_model(df_test, model = xgb2):
    df = df_test.copy()
    tr = 0.4
    
    prob = model.predict_proba(df)[:][: , 1]
    pred = np.where(prob > tr, 1, 0)
    return pred

We'll see how much of last year customer that's actually potential, but because they didn't get pitched by the company, they didn't take the product.

In [59]:
filter1 = df['TypeofContact'] == 'Self Enquiry'
filter2 = df['ProdTaken'] == 0
df_test = df[filter1 & filter2]
df_test.head()

In [60]:
df_testx = data_processing(df_test)
df_test['ProdTakenpred'] = last_model(df_test = df_testx)
df_test['ProdTaken_Pot'] = df_test['ProdTakenpred'].apply(lambda x: "Potential" if x == 1 else "Not potential")

In [61]:
g = df_test.ProdTaken_Pot.value_counts().plot(kind = 'barh')

for p in g.patches:
    g.annotate(format(p.get_width(), '0'),
               xy = (p.get_x() + p.get_width() / 2,
                     p.get_y() + p.get_height() / 2,),
               ha = 'center',
               va = 'center',
               color = 'white'
              )

Based on the plot, we actually have 530 potential customer that is predicted will take the product if pitched by our company.